In [3]:
"""We didnt have time to correct minor differences between the kaggle dataset and recent dataset, so we  
were not able to use the same notebook for the analysis of both sets. 
As such, there is a lot of code repetition between this notebook and Analyse_Data_Recent.ipynb"""

import pandas as pd

In [4]:
videos_kaggle = pd.read_csv("Videos_merged.csv")

#define year and month

def define_year(row):
    trending_date = row["trending_date"]
    return trending_date.split(".")[0]

def define_month(row):
    trending_date = row["trending_date"]
    return trending_date.split(".")[2]
    
videos_kaggle["year"] = "20"+ videos_kaggle.apply(define_year, axis=1)
print(videos_kaggle["year"].value_counts())

videos_kaggle["month"] = videos_kaggle.apply(define_month, axis=1)
print(videos_kaggle["month"].value_counts())



year
2018    123043
2017     38386
Name: count, dtype: int64
month
12    24791
03    24628
05    23521
01    23180
02    22269
04    18787
11    13595
06    10658
Name: count, dtype: int64


In [5]:
"""Identify the categories with least expression and change its category to 'Other', in each group of country, 
category, year and month"""

def categories_to_merge(df):
    # rank each group and return the categories with the least expression
    df = df.rank(axis=0, pct=True).reset_index()
    return list(df[df["aggregation"] < 0.3]["category"].unique())

def assign_category_other(row,list_categories):
    # change the category to 'Other' if the original category is in the list
    if str(row["category"]) in list_categories:
        return "Other" 
    else:
        return row["category"]

def build_table(df,column,how):
    df_aux = df.copy()
    
    # group by country, category, year and month. 
    # the argument 'column' gives us the column we want to aggregate.
    # the argument 'how' gives us the type of aggregation (sum, count...)
    output = df_aux.groupby(["country","category", "year", "month"]).agg({column:how})
    output.rename(columns = {column:"aggregation"}, inplace = True)

    # analyse the minoritary categories and change those categories to 'Other'
    list_categories_to_merge = categories_to_merge(output.copy())
    print("Minoritary categories: " + ",".join(list_categories_to_merge))
    output.reset_index(inplace=True)
    df_aux["category"] = df_aux.apply(assign_category_other,axis=1,args=(list_categories_to_merge,))

    # group again by country, category, year and month
    output = df_aux.groupby(["country","category", "year", "month"]).agg({column:how})
    output.rename(columns = {column:"aggregation"}, inplace = True)
    
    # pivot the table so we have a column per category
    output = output.pivot_table(index=['country', 'year','month'], columns='category')
    output.to_csv(column+"_"+how+"_raw.csv")
    
    return output




In [6]:
# Build the table for category (agg = count)
build_table(videos_kaggle,"category","count")


Minoritary categories: Autos & Vehicles,Education,Movies,Pets & Animals,Science & Technology,Shows,Trailers,Travel & Events,Gaming,Howto & Style,News & Politics,Nonprofits & Activism,Sports


aggregation                                              
category                Comedy Entertainment Film & Animation Music Other   
country year month                                                          
DE      2017 11            215          1134              248   129  1104  \
             12            404          2416              408   345  1623   
        2018 01            347          2133              333   310  1780   
             02            308          2041              320   297  1752   
             03            357          2290              336   351  1948   
             04            298          1672              237   344  1469   
             05            422          2463              346   419  1616   
             06            183          1143              148   177   730   
FR      2017 11            453           784              197   229  1290   
             12            788          1533              351   516  2092   
        2018 01            622          1307              306   570  2199   
             02            482          1338              307   573  2060   
             03            582          1410              311   646  2217   
             04            516          1073              228   534  1736   
             05            621          1603              321   617  2125   
             06            279           771              136   261   907   
GB      2017 11            200           772              104   833  1168   
             12            348          1382              297  1580  1844   
        2018 01            252          1376              306  1648  1642   
             02            326          1269              495  1722  1258   
             03            250          1458              473  2404  1199   
             04            168          1024              343  2154   523   
             05            178          1296              381  2353   639   
             06            106           547              178  1060   344   
US      2017 11            315           790              174   613  1261   
             12            579          1549              360   950  2244   
        2018 01            465          1288              354   758  2464   
             02            423          1243              295   777  2391   
             03            524          1537              337   810  2467   
             04            407          1160              264   747  1751   
             05            510          1701              355  1121  2111   
             06            234           696              206   696   812   

                                   
category           People & Blogs  
country year month                 
DE      2017 11               558  
             12               967  
        2018 01               842  
             02               805  
             03               858  
             04               741  
             05               844  
             06               373  
FR      2017 11               428  
             12               887  
        2018 01               764  
             02               783  
             03               971  
             04               682  
             05               820  
             06               384  
GB      2017 11               321  
             12               734  
        2018 01               557  
             02               490  
             03               325  
             04               197  
             05               187  
             06               115  
US      2017 11               247  
             12               518  
        2018 01               467  
             02               442  
             03               508  
             04               471  
             05               401  
             06               156

In [7]:
# Build the table for views (agg = sum)
build_table(videos_kaggle,"views","sum")


Minoritary categories: Autos & Vehicles,Education,Gaming,Howto & Style,Movies,News & Politics,Pets & Animals,Science & Technology,Shows,Trailers,Travel & Events,Nonprofits & Activism


aggregation                                               
category                Comedy Entertainment Film & Animation        Music   
country year month                                                           
DE      2017 11      141037572     419301881        194680306    321904208  \
             12      293917676    1667292021        181440810    557761948   
        2018 01      220354912     814353108        175569694    749790520   
             02      172224220    1034922137        298789765    560550168   
             03      219331325    1138466867        296700649    862409052   
             04      188578968     960371438        157798382    808678002   
             05      303147802    1337014082        274894553   1293625532   
             06      103565679     730917160        177310177    529151216   
FR      2017 11      191066111     194913199         52101125    350802421   
             12      401462567     648952852         86573647    498844989   
        2018 01      241300753     349461018         81050864    714627288   
             02      203634157     525964258        162935907    525547736   
             03      218585668     538382315        176891887    707602425   
             04      212419975     424792471         72228432    797061714   
             05      311976027     717642547        194246343    964928834   
             06      115839391     379391569         92488813    467032115   
GB      2017 11      264224050    1285179140        155511937   4158943353   
             12      924400764    5848482923        317152438   5427655009   
        2018 01      544667482    2108655386        253571983   8102315782   
             02      567146563    4904719539       1756239356  20079545911   
             03      465085349    4458132401       2229906959  43583337914   
             04      454240408    4102582393       1186340865  28575563233   
             05      441242528    4599479825       1865698212  46662717236   
             06      301054999    2479051434        598285876  14570786331   
US      2017 11      311563737     831707678        439446191   1814866039   
             12      645468359    3054867831        347868839   2234885386   
        2018 01      620505089    1291889488        478759602   2034296336   
             02      452847312    1542021267       1002360186   1865874026   
             03      794689372    2667651676        857868920   3912721269   
             04      913708890    3149258988        949212518   6359916819   
             05      841677002    5474175011       1752234656  15023340400   
             06      536966447    2592816256       1456405809   6886991915   

                                                           
category                 Other People & Blogs      Sports  
country year month                                         
DE      2017 11      208155145      148534546   129211676  
             12      391559334      281309390   222151210  
        2018 01      359686495      210699546   240986849  
             02      463847484      460117847   303316061  
             03      409172648      306812086   352375631  
             04      346765174      251284537   247536865  
             05      724294266      346873326   287094304  
             06      264815155      187981990   155519682  
FR      2017 11      161486036       54780928   110888260  
             12      263674162      156096070   238418976  
        2018 01      235511312      100222322   306696670  
             02      305121235      273823103   300373372  
             03      209752244      161273822   326412892  
             04      243426520      152496576   250139311  
             05      535555525      236356704   291605653  
             06      188476284      145794008   172275905  
GB      2017 11      711150126      114619489   101228968  
             12      647782719      687830599   150245144  
        2018 01      7028877

In [8]:
# Build the table for comment_count (agg = sum)
build_table(videos_kaggle,"comment_count","sum")


Minoritary categories: Autos & Vehicles,Education,Howto & Style,Movies,News & Politics,Pets & Animals,Science & Technology,Shows,Trailers,Travel & Events,Film & Animation,Gaming,Nonprofits & Activism


aggregation                                     
category                Comedy Entertainment     Music     Other   
country year month                                                 
DE      2017 11         766246       1619766   2512425   1681235  \
             12        1665228      10126603   2325548   2677915   
        2018 01        1410919       4247372   2993503   2889360   
             02         810771       4092326   2551069   3649110   
             03        1273347       4970506   3281805   2756982   
             04         879566       3545055   3915309   1994089   
             05        1303179       5212912   9361330   4073917   
             06         742713       2672677   2680847   2070444   
FR      2017 11         920032        814138   2098886    925727   
             12        1666870       4536813   2009218   1528395   
        2018 01        1057634       1544332   3005043   2087079   
             02         942699       1813970   2250458   1865559   
             03         969165       2081357   3729658   1502867   
             04        1433076       1229875   3527924   1608063   
             05        1433301       2422197   6753684   2876934   
             06         714037       1287242   2071418   1388324   
GB      2017 11        1662071       3480115  12811560   3184327   
             12        3267388      29350416  18968009   4518697   
        2018 01        5050814      26494330  18404527   5989472   
             02        2453346       8247556  30637674   9852342   
             03        1525026      13924916  54675294   9974465   
             04        1468823      16167349  46765182   3891883   
             05         901598      13282850  84463756   5920210   
             06         389750       5949908  26287138   2943394   
US      2017 11        1564253       3385777   8210654   3762277   
             12        3076674      15621876   8535309   7442207   
        2018 01        3444270      11889648   7777954  12612852   
             02        1354489       4305475   5776652   9244820   
             03        3067956      11228281  15889288  11970295   
             04        3408557       7932261  17386222  10191374   
             05        4078069      12098933  44921934  20729987   
             06        2551314       7104247  16798383  12555689   

                                            
category           People & Blogs   Sports  
country year month                          
DE      2017 11            665790   297924  
             12           1266387   473776  
        2018 01            942328   551130  
             02           1401643   700153  
             03           1024960   797569  
             04            882372   437194  
             05           1255681   603789  
             06           1480039   358850  
FR      2017 11            165903   297322  
             12            536792   536633  
        2018 01            371075   776037  
             02            475796   719503  
             03            355340   685795  
             04            460646   397234  
             05            788298   765706  
             06            950968   397188  
GB      2017 11            330070   187419  
             12           5718861   342164  
        2018 01           3057983   373723  
             02           3071193  2049591  
             03            994524  5940369  
             04           1229579   727511  
             05           2908564   246519  
             06           1402375   188930  
US      2017 11            541418   155833  
             12           2837888   602915  
        2018 01           1618080   863110  
             02           2901981  1529135  
             03           1826559  2384523  
             04           4690633  1871021  
             05           5061687  3043530  
             06           5299786   742088

In [9]:
# Build the table for likes (agg = sum)
build_table(videos_kaggle,"likes","sum")


Minoritary categories: Autos & Vehicles,Education,Howto & Style,Movies,News & Politics,Pets & Animals,Science & Technology,Shows,Trailers,Travel & Events,People & Blogs,Gaming,Nonprofits & Activism,Sports


aggregation                                             
category                Comedy Entertainment Film & Animation      Music   
country year month                                                         
DE      2017 11        6068565      14171844          4531410   20845239  \
             12       16270192      60876128          4538504   29387783   
        2018 01       10952858      29783610          5926501   35212062   
             02        7113500      32360666          7758378   33157420   
             03        9098110      37597871          8048785   35487800   
             04        8278922      31200137          3689010   44940187   
             05       11899650      38944527          7085924   72370872   
             06        5105608      22817489          3737034   28895703   
FR      2017 11       13714303       7816131          1453532   20960023   
             12       29925754      23364661          1793309   26780170   
        2018 01       16151814      13076856          2878345   35863685   
             02       14757342      15353385          4065732   30650082   
             03       14074133      18137064          5129913   37977332   
             04       15919058      11485672          1950450   41819736   
             05       18649746      17638633          5179406   57954396   
             06        7613294      11371158          2180735   25308810   
GB      2017 11       11020364      35794190          3465578  147911597   
             12       38822892     161959660          4635690  204021786   
        2018 01       22243272      74972959          4907582  254387481   
             02       16484803      70341909         33177490  398350495   
             03       17550720     114331171         43229706  801781698   
             04       17102707     130022930         20308472  617290545   
             05       12273147     105568548         28499255  977519883   
             06        7874271      51274864          9194860  341729567   
US      2017 11       14335435      26570394          8291683   85173641   
             12       35337729      92625755          7213793   98943833   
        2018 01       29575644      35461787         13185882   97809472   
             02       15744798      30031618         22304634   80661436   
             03       35059632      82203203         19915566  162284475   
             04       33396496      75221354         23162859  212714773   
             05       33133699     129247972         41645661  451793906   
             06       19763313      59154408         30277398  227457048   

                               
category                Other  
country year month             
DE      2017 11      13223836  
             12      27717690  
        2018 01      23608230  
             02      27290955  
             03      25648391  
             04      21272288  
             05      31477354  
             06      20403552  
FR      2017 11      11013822  
             12      21158320  
        2018 01      21614690  
             02      20363963  
             03      18176426  
             04      16643718  
             05      26685098  
             06      15506009  
GB      2017 11      23042368  
             12      47696399  
        2018 01      50944578  
             02      87996662  
             03      97802173  
             04      66259148  
             05      39831649  
             06      18887259  
US      2017 11      24661225  
             12      63245961  
        2018 01      75849655  
             02      69153821  
             03     103346386  
             04      98013074  
             05     172325295  
             06     104852484